Continuous Training in Machine Learning with Azure MLOps: Wine Quality Dataset
Student Version
=======================================================================

COURSE: Introduction to MLOps with Azure ML <br>
ASSIGNMENT: Implementing Continuous Training with Data Drift Detection <br>

LEARNING OBJECTIVES: <br>
- Understand the concept of continuous training in machine learning<br>
- Implement data drift detection using a discriminator approach<br>
- Create an end-to-end MLOps pipeline in Azure ML<br>
- Practice real-world machine learning deployment techniques<br>
- Apply best practices for maintaining ML models in production<br>

INSTRUCTIONS:<br>
- Complete all sections marked with TODO or EXERCISE<br>
- Return the notebook with the cell outputs

In [ ]:
!pip install -r requirements.txt

# Continuous Training in Machine Learning with Azure MLOps


Setup and Dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, mean_squared_error, r2_score
import datetime
import joblib
import os
import json
import requests
from io import StringIO

# Create directories for saving models and data
os.makedirs("./models", exist_ok=True)
os.makedirs("./data", exist_ok=True)

# SECTION 1: Connecting to Azure ML Workspace
---
📝 EXERCISE (1 point): Create an Azure ML workspace and export its config file in this directory

*N.B. : Cet exercise n'est pas nécessaire pour le reste du cours, mais il est pratique pour l'exercise bonus.*

In [ ]:
def connect_to_workspace():
    """Connect to Azure ML workspace using parameters."""

    ws = Workspace.get(
       name="YOUR_WORKSPACE_NAME",
       subscription_id="YOUR_SUBSCRIPTION_ID",
       resource_group="YOUR_RESOURCE_GROUP"
    )

# Connect to workspace
ws = connect_to_workspace()
ws

# SECTION 2: Loading and exploring the wine quality dataset

In [ ]:
def load_wine_quality_data():
    """
    Load Wine Quality dataset from UCI repository.
    This demonstrates data acquisition in a production ML pipeline.
    """
    # Download the data from the UCI repository
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    print("Attempting to download Wine Quality dataset from UCI repository...")
    response = requests.get(url)
    if response.status_code == 200:
        data = pd.read_csv(StringIO(response.text), sep=';')
        print("Wine Quality dataset loaded successfully!")
        
        # Save the raw data for future reference - good practice in ML pipelines
        data.to_csv("./data/winequality-raw.csv", index=False)
    else:
        raise Exception(f"Failed to download dataset. Status code: {response.status_code}")
    
    # Add timestamp to simulate real-world data collection
    # This is important for time-series analysis and continuous training
    current_date = datetime.datetime.now()
    dates = [current_date - datetime.timedelta(days=i) for i in range(len(data))]
    data['timestamp'] = dates
    
    return data

# Load the dataset
wine_data = load_wine_quality_data()
wine_data['good_quality'] = (wine_data['quality'] >= 6).astype(int)


print(f"Wine Quality dataset shape: {wine_data.shape}")

# Display the first few rows and dataset info
print("\nFirst 5 rows of the dataset:")
print(wine_data.head())

print("\nDataset information:")
print(wine_data.info())

print("\nSummary statistics:")
print(wine_data.describe())

We are now creating a  visualization to explore the distribution of wine quality

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(wine_data['quality'], kde=True, discrete = True)
plt.title('Distribution of Wine Quality')
plt.xlabel('Quality Score')
plt.ylabel('Count')
plt.axvline(x=5.5, color='red', linestyle='--', label='Good Quality Threshold')
plt.legend()
plt.show()

We can see the data is well balanced between "good" and "bad" wines <br>
Let's create a correlation matrix heatmap to explore relationships <br>

In [ ]:
# We dont need the quality anymore 
wine_data.drop(columns=['quality'], inplace=True)


plt.figure(figsize=(12, 8))
correlation_matrix = wine_data.drop(['timestamp', 'good_quality'], axis=1).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Wine Features')
plt.tight_layout()
plt.show()

# SECTION 3: Initial model training. <br>
The initial model is usually called the champion model <br>
In a real-world environnement, this would be an already registered model.pkl model in Azure. <br>
We are training this model with original data, then the simulated new data will have some applied drift.

In [ ]:
# Prepare data for training
X = wine_data.drop(['good_quality', 'timestamp'], axis=1)
y = wine_data['good_quality']  # Using the binary classification target

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def train_model(X_train, y_train, **kwargs):
    """
    Train a model on the given data.
    
    Args:
        X_train: Training features
        y_train: Training target    
    Returns:
        Trained model object
    """
    
    model = RandomForestClassifier(random_state=42, **kwargs)
    
    # Fit the model on training data
    model.fit(X_train, y_train)
    print("Model training complete!")
    
    return model

def evaluate_classifier(model, X_test, y_test):
    """Evaluate a classification model and return performance metrics."""
    # Get probability predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate AUC-ROC - good for imbalanced classification
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Calculate accuracy - simple but can be misleading for imbalanced data
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Model AUC-ROC: {auc:.4f}")
    print(f"Model Accuracy: {accuracy:.4f}")
    
    return auc, accuracy


initial_model = train_model(X_train, y_train, n_estimators=100)
initial_auc, initial_accuracy = evaluate_classifier(initial_model, X_test, y_test)

Take notes of the current model's performance. It will be important when compared to drifted data.

Let's view what are the most important features for this model.

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': initial_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance for Wine Quality Prediction')
plt.tight_layout()
plt.show()

print("\nTop 5 most important features:")
print(feature_importance.head())


# Saving the initial model
# In production, you would register this model in Azure ML
initial_model_path = "./models/initial_model.pkl"
joblib.dump(initial_model, initial_model_path)
print(f"Initial model saved to {initial_model_path}")


# SECTION 4: Data drift detection using discriminator approach <br>
This section demonstrates how to detect data drift, which is critical for continuous training <br>
There are multiple ways to choose when to retrain : periodically (e.g. every 1 week), when model performance drops bellow a threshold, or when data drift is detected.


We will implement here a continuous training pipeline triggered when drift is detected, with a discriminator approach. 

The discriminator approach is as follows : <br>
Between two datasets, if there is drift, a second model (discriminator) should be able to tell the datasets appart. <br>

Thus, the target column is not the quality of the wine, but its source : historical / new dataset. <br>
If the discriminator has good scores, then drift is present and the model should be retrained.



In [ ]:
def create_drift_wine_data(original_data, drift_percentage=0.3, magnitude=0.5):
    """
    Create a drifted version of the wine dataset.
    
    Args:
        original_data: Original wine dataset
        drift_percentage: Percentage of features to apply drift to
        magnitude: Magnitude of the drift
    
    Returns:
        DataFrame with drifted values
    """
    drift_data = original_data.copy()
    
    # Select a subset of features to drift (excluding target and timestamp)
    features = [col for col in drift_data.columns if col not in ['quality', 'good_quality', 'timestamp']]
    num_features_to_drift = max(1, int(len(features) * drift_percentage))
    features_to_drift = np.random.choice(features, num_features_to_drift, replace=False)
    
    # Apply drift to selected features
    for feature in features_to_drift:
        # Get feature statistics
        mean_val = drift_data[feature].mean()
        std_val = drift_data[feature].std()
        
        # Apply a selective reversal drift - this inverts the relationship 
        # between feature and target for more extreme values
        threshold = drift_data[feature].quantile(0.6)  # Top 40% of values
        mask = drift_data[feature] > threshold
        
        # For high values: reflect across the threshold with amplification
        drift_data.loc[mask, feature] = threshold - (1.5 * magnitude) * (drift_data.loc[mask, feature] - threshold)
        
        # For normal values: add non-linear component
        drift_data.loc[~mask, feature] = drift_data.loc[~mask, feature] + magnitude * np.square(drift_data.loc[~mask, feature] - mean_val) / std_val
    
    # Create a newer timestamp for the drift data
    latest_date = original_data['timestamp'].max()
    new_dates = [latest_date + datetime.timedelta(days=i+1) for i in range(len(drift_data))]
    drift_data['timestamp'] = new_dates
    
    print(f"Drift applied to features: {', '.join(features_to_drift)}")
    return drift_data

# Generate drift data from a subset of the original wine data
# We'll use 30% of the original data to create the drift dataset
np.random.seed(42)  # For reproducibility
drift_indices = np.random.choice(
    wine_data.index, 
    size=int(0.3 * len(wine_data)), 
    replace=False
)
subset_for_drift = wine_data.loc[drift_indices].copy()
drift_data = create_drift_wine_data(subset_for_drift, drift_percentage=0.4, magnitude=0.8)

print(f"New data with drift shape: {drift_data.shape}")

In [ ]:
# Show at least 4 features in a 2x2 subplot grid

plt.figure(figsize=(15, 10))
drift_features = [col for col in drift_data.columns if col not in ['quality', 'good_quality', 'timestamp']][:4]

for i, feature in enumerate(drift_features):
    plt.subplot(2, 2, i+1)
    sns.kdeplot(wine_data[feature], label='Original Data', fill=True, alpha=0.3)
    sns.kdeplot(drift_data[feature], label='Drift Data', fill=True, alpha=0.3)
    plt.title(f'Distribution of {feature}')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# The function should label the original data as 0 and new data as 1, then combine them

def prepare_discriminator_data(original_data, new_data):
    """
    Prepare data for training a discriminator model to detect drift.
    
    Args:
        original_data: Original dataset
        new_data: New dataset that may have drift
        
    Returns:
        X: Combined features from both datasets
        y: Binary labels (0 for original, 1 for new)
    """
    # Get features only - exclude target and timestamp
    original_features = original_data.drop(['good_quality', 'timestamp'], axis=1)
    new_features = new_data.drop(['good_quality', 'timestamp'], axis=1)
    
    # Label the data sources: 0 for original, 1 for new
    original_features['source'] = 0
    new_features['source'] = 1
    
    # Combine the datasets
    combined_data = pd.concat([original_features, new_features], axis=0)
    
    # Shuffle the data for better training
    combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Split features and label
    X = combined_data.drop('source', axis=1)
    y = combined_data['source']
    
    return X, y

# Prepare data for the discriminator
X_disc, y_disc = prepare_discriminator_data(wine_data, drift_data)
X_disc_train, X_disc_test, y_disc_train, y_disc_test = train_test_split(X_disc, y_disc, test_size=0.2, random_state=42)

In [ ]:
# Train the discriminator model
discriminator_model = RandomForestClassifier(n_estimators="A REMPLIR", random_state=42)
discriminator_model.fit(X_disc_train, y_disc_train)

# Evaluate the discriminator
y_disc_pred_proba = discriminator_model.predict_proba(X_disc_test)[:, 1]
discriminator_auc = roc_auc_score(y_disc_test, y_disc_pred_proba)
print(f"Discriminator AUC-ROC: {discriminator_auc:.4f}")

We can see it is **extremely easy** for the model to discriminate between the two datasets. It means our discriminator can easily guess where the data comes from. If the data was undrifted, the discriminator wouldnt be able to guess its source <br>

In a real world scenario, the AUC-ROC metric can be used as a trigger to retrain a model.

# SECTION 5: Continuous Training Decision Logic <br>
We will now start to build brick by brick the continuous machine learning pipeline.

We first need a quick function to decide wether retraining is needed or not.

In [ ]:
def should_retrain(discriminator_auc, threshold=0.7):
    """
    Decide if retraining is needed based on the discriminator's ability to distinguish data sources.
    
    Args:
        discriminator_auc: AUC-ROC score of the discriminator model
        threshold: Threshold above which we consider significant drift detected
        
    Returns:
        bool: True if retraining is recommended
    """
    if discriminator_auc >= threshold:
        print(f"Significant data drift detected (AUC = {discriminator_auc:.4f} > {threshold}).")
        print("Recommendation: Retrain the model with new data.")
        return True
    else:
        print(f"No significant data drift detected (AUC = {discriminator_auc:.4f} <= {threshold}).")
        print("Recommendation: Continue using the current model.")
        return False

# Check if retraining is recommended based on the discriminator performance
# retrain_recommended = should_retrain(discriminator_auc, threshold=0.7)

# For teaching purposes, let's assume a high discriminator AUC (simulating drift)
# In reality, this value would come from the actual discriminator model
retrain_recommended = should_retrain(discriminator_auc, threshold=0.7)


When data drift is detected, a proper workflow would track the drift. Here, we have a discriminator mode. To track where the drift comes from, we could simply look at what features were the most important to the discirminator model. If a feature is extremely important for the discriminator, data drift occured there.

In [ ]:
# Create a visualization of the features that show the most drift according to the discriminator

disc_feature_importance = pd.DataFrame({
    'Feature': X_disc.columns,
    'Importance': discriminator_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=disc_feature_importance)
plt.title('Feature Importance for Drift Detection')
plt.tight_layout()
plt.show()


print("\nTop features that show the most drift:")
print(disc_feature_importance.head()

We can see the important features for the discriminator are the ones we applied a drift to :  `pH, alcohol, sulphates, chlorides` 

# SECTION 6: Model Retraining <br>
If drift is detected, we retrain the model with combined data <br>


In [ ]:
# Retrain the model when drift is detected
# The model should be trained on both original and new data

if retrain_recommended:
    # Using only the drifted data for retraining
    X_drifted = drift_data.drop(['good_quality', 'timestamp'], axis=1)
    y_drifted = drift_data['good_quality']
    
    # Prepare for retraining
    X_drift_train, X_drift_test, y_drift_train, y_drift_test = train_test_split(
        X_drifted, y_drifted, test_size=0.2, random_state=42
    )
    
    # Retrain the model on only the drifted data
    retrained_model = train_model(X_drift_train, y_drift_train, n_estimators=100)
    print("Model retrained with drifted data only!")
    
    # Evaluate the retrained model on drifted test data
    y_retrain_pred_proba = retrained_model.predict_proba(X_drift_test)[:, 1]
    retrained_auc = roc_auc_score(y_drift_test, y_retrain_pred_proba)
    print(f"Retrained model AUC-ROC on drift test data: {retrained_auc:.4f}")
    
    # Compare with original model on the new test data
    original_on_drift_auc = roc_auc_score(
        y_drift_test, 
        initial_model.predict_proba(X_drift_test)[:, 1]
    )
    print(f"Original model AUC-ROC on drift test data: {original_on_drift_auc:.4f}")
    
    # Visualization of model comparison
    plt.figure(figsize=(8, 6))
    
    # Calculate ROC curve for both models
    fpr_original, tpr_original, _ = roc_curve(y_drift_test, initial_model.predict_proba(X_drift_test)[:, 1])
    fpr_retrained, tpr_retrained, _ = roc_curve(y_drift_test, y_retrain_pred_proba)
    
    # Plot ROC curves
    plt.plot(fpr_original, tpr_original, label=f'Original Model (AUC = {original_on_drift_auc:.4f})')
    plt.plot(fpr_retrained, tpr_retrained, label=f'Retrained Model (AUC = {retrained_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison: Original vs Retrained Model')
    plt.legend()
    plt.show()
    
    # Save the retrained model
    retrained_model_path = "./models/retrained_model.pkl"
    joblib.dump(retrained_model, retrained_model_path)
    
    print("Model retraining completed!")

else:
    print("No retraining needed. The current model is still valid.")

Here we can see the retraining was justified on the drifted data. Theold model was performing very poorly on the drifted dataset. 

--- 

📝 EXERCISE (1 point): Why shouldn't we use all combined data ? How may it be a problem for the model's performance. In ~ 50 words

ANSWER : Using all data means learning on values that are not up to date if the drift is permanent. The learned feature interactions of old data is now noise to the model. But learning only on drifted data means learning on a smaller dataset, impacting the model's performance.

# SECTION 7: Azure ML Pipeline for Continuous Training

### **Pipeline Overview**  
This pipeline automates detecting changes in data patterns (drift) and retraining our model if needed, then registering it as the new production model. It can be scheduled to run every week, for instance. Let's assume we have a data stream that updates every week the current and historical data.



It uses **three inputs**:  
- `model.pkl` (a pre-trained model from Azure)  
- `current_data.csv` (new data to analyze)  
- `historical_data.csv` (past data for comparison)  


It produces **two outputs**:  
- Updated `model.pkl` (if retraining occurs)  
- A `json` file indicating whether drift was detected .  

### **Step-by-Step Workflow**  
You will see in the `components` subfolder the different components to add to Azure ML Designer. They are based on the previous work.
1. **Data Preparation**  
   - Clean and format `historical_data.csv` by removing the `quality` column. 

2. **Drift Detection**  
   - This step compares `current_data.csv` with `historical_data.csv` to check if significant differences exist, with the help of a discriminator model. If drift is found (`drift_detected = True`), the model will not be registered.

3. **Model Training**  
   - If drift is detected, train a new model using the updated data. This step ensures the model adapts to new patterns in the data
   - This step is done in parallel to drift detection in order to save overall time, but a proper workflow would only train the model if drift was detected.

4. **Registering Model (if drift was detected)**  
   - If drift was detected, save the newly trained model (`model.pkl`) to Azure as the latest version. This makes it available for future deployments or pipelines.
   - The registered model.pkl will be in the blob storage.

--- 

📝 EXERCISE (2 points): Create Azure ML Components via Web Interface

TODO: Follow these steps in the Azure ML Studio web interface:

1. Create a new Dataset:
   - Go to 'Assets' > 'Datasets' > 'Create dataset'
   - Upload winequality-raw.csv from your local files
   - Name it 'wine_quality_training_data'
   - Create a second dataset with drifted_data.csv
2. Create Compute Target:
   - Go to 'Manage' > 'Compute' > 'Compute clusters' > 'New'
   - Create CPU cluster named 'training-cluster' with:
     - VM size: Standard_DS3_v2
     - Minimum 0, maximum 2 nodes
     - Idle seconds before scale down: 120
     - Wait a few minutes
3. Create Pipeline Components:
   - Go to 'Assets' > 'Components' > 'Create new'
   - Create these components through the UI:
     - data_preparation Component
     - drift_detection Component
     - model_training Component
     - model_registration Component

--- 
📝 EXERCISE (2 points): Configure Pipeline via Web Interface
TODO: Create pipeline using drag-and-drop interface
1. Go to 'Authoring' > 'Designer' > Create a pipeline using custom components
2. Drag in the 4 created components. (Note : The drifted data shouldnt be prepared, it already has the "good_quality" column.)
3. Connect them in sequence
4. Configure schedule:
   - Set to run weekly using trigger configuration
   - Enable email notifications for pipeline failures
5. Publish pipeline as 'wine_quality_ct_pipeline'

--- 
📝 BONUS EXERCISE (1 point): Launch this pipeline using azure cli and paste the lines here

You will need to create a pipeline.yaml file describing the different step you have done in the UI

QUESTIONS :

Question 1: Expliquez le concept de "continuous training" en Machine Learning et pourquoi il est essentiel dans un environnement de production. Donnez au moins 3 raisons principales.

Question 2: Qu'est-ce que le data drift et comment peut-il affecter les performances d'un modèle de machine learning en production ? Illustrez avec un exemple concret du domaine viticole.

Question 3: Décrivez en détail l'approche discriminateur pour détecter le data drift. Comment fonctionne cette méthode et que signifie un AUC-ROC élevé (>0.7) pour le modèle discriminateur ?

Question 4: Dans le TP, le discriminateur obtient un AUC-ROC très élevé. Analysez ce résultat : qu'est-ce que cela indique sur la qualité des données ? Quel seuil d'AUC-ROC recommanderiez-vous pour déclencher un réentraînement ?

Question 5: Quels sont les avantages de créer des composants réutilisables dans Azure ML Designer par rapport à un script monolithique ? Donnez au moins 4 avantages pratiques pour une équipe de data scientists.